# Sequence Classification using Recurrent Neural Networks(RNN)
In this homework, you will learn how to train a recurrent neural network for human action classification. RNN is designed handle sequential data. The network can incorporate both past history and current input. [This](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) is a very good tutorial. You should read it before you start.

## Setup
Please make sure you have h5py and torchnet installed
> pip install h5py

> pip install git+https://github.com/pytorch/tnt.git@master


In [2]:
import os
import numpy as np
import h5py

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as DD
import torchnet as tnt

use_cuda = torch.cuda.is_available()
print('use cuda: %s'%(use_cuda))
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if use_cuda else torch.ByteTensor


use cuda: True


## Dataset
The data we are using is skeleton data, which indicates the 3D locations of body joints. In total, there are 25 body joints. It is collected by Kinect v2. To make it easier, each sequence have same number of frames. You need to classify 10 different actions. There are 2000 training sequences, 400 validation sequences, and 500 test sequences. Each sequence has 15 frames, each frame is a 75-dimension vector (3*25).

For your convenience, we provide the dataloader for you.


In [4]:
class Dataset(DD.Dataset):
    # subset can be: 'train', 'val', 'test'
    def __init__(self, data_path, subset='train'):
        super(Dataset, self).__init__()
        self.data_path = os.path.join(data_path, '%s_data.h5'%subset)
        self.subset = subset

        with h5py.File(self.data_path) as f:
            self.data = np.array(f['data'])

        if subset != 'test':
            self.label_path = os.path.join(data_path, '%s_label.h5'%subset)
            with h5py.File(self.label_path) as f:
                self.label = np.array(f['label'])

        self.num_sequences = self.data.shape[0]
        self.seq_len = self.data.shape[1]
        self.n_dim = self.data.shape[2]

    def __getitem__(self, index):
        seq = self.data[index]
        if self.subset != 'test':
            label = int(self.label[index])
            sample = {'seq': seq, 'label': label}
        else:
            sample = {'seq': seq}
        return sample

    def __len__(self):
        return self.num_sequences

trSet = Dataset('./data', subset='train')
valSet = Dataset('./data', subset='val')
tstSet = Dataset('./data', subset='test')

batch_size = 50
trLD = DD.DataLoader(trSet, batch_size=batch_size, sampler=DD.sampler.RandomSampler(trSet), 
                     num_workers=4, pin_memory=False)

valLD = DD.DataLoader(valSet, batch_size=batch_size, sampler=DD.sampler.SequentialSampler(valSet),
                      num_workers=4, pin_memory=False)

tstLD = DD.DataLoader(tstSet, batch_size=batch_size, sampler=DD.sampler.SequentialSampler(tstSet),
                      num_workers=4, pin_memory=False)

input_dim = trSet.n_dim
num_class = 10

In [15]:
trSet.data.shape

(2000, 15, 75)

## Model
Pytorch has implemented different types of recurrent layers for you. For this homework, you can use any type of RNNs as you want:
> torch.nn.RNN()

> torch.nn.LSTM()

> torch.nn.GRU()

You can check details for different types of recurrent layers here: [RNN](http://pytorch.org/docs/master/nn.html#torch.nn.RNN), [LSTM]( http://pytorch.org/docs/master/nn.html#torch.nn.LSTM), [GRU](http://pytorch.org/docs/master/nn.html#torch.nn.GRU)


### Implement a specific model
In this section, you need to implement a model for sequence classification. The model has following layers:
* A linear layer that can map features of 75-dimension to 100-dimension.
* 1 Layer LSTM layer with hidden size of 100
* A linear layer that goes from 100 to num_class (10). 

An LSTM layer takes an input of size of (batch_size, seq_len, fea_dim) and outputs a variable of shape (batch_size, seq_len, hidden_size). In this homework, the classification score for a sequence is the classification score for the last step of rnn_outputs.



In [5]:
# sequence classification model
class SequenceClassify(nn.Module):
    def __init__(self):
        super(SequenceClassify, self).__init__()
        
        ############## 1st To Do (10 points) ##############
        ###################################################
        self.project_layer = nn.Linear(75, 100)
        self.recurrent_layer = nn.LSTM(input_size=100, hidden_size=100)
        self.classify_layer = nn.Linear(100, 10)
        ###################################################
    
    # the size of input is [batch_size, seq_len(15), input_dim(75)]
    # the size of logits is [batch_size, num_class]
    def forward(self, input, h_t_1=None, c_t_1=None):
        # the size of rnn_outputs is [batch_size, seq_len, rnn_size]
        rnn_outputs, (hn, cn) = self.recurrent_layer(self.project_layer(input))
        # classify the last step of rnn_outpus
        # the size of logits is [batch_size, num_class]
        logits = self.classify_layer(rnn_outputs[:, -1])
        return logits

model = SequenceClassify()
model.cuda()

SequenceClassify(
  (project_layer): Linear(in_features=75, out_features=100, bias=True)
  (recurrent_layer): LSTM(100, 100)
  (classify_layer): Linear(in_features=100, out_features=10, bias=True)
)

## Train the model
After you have the dataloader and model, you can start training the model. Define a SGD optimizer with learning rate of 1e-3, and a cross-entropy loss function:

In [9]:
################ 2nd To Do  (5 points)##################
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()

In [10]:
for batch_idx, sample in enumerate(trLD):
    sequence = sample['seq']
    label = sample['label']
    print(batch_idx)
    print(sequence.size())
    print(label.size(), '\n')
    print(sequence[:, -1], '\n')
    print(sequence[:, -1, :], '\n')
    print(sequence[:, -1].size())
    break

0
torch.Size([50, 15, 75])
torch.Size([50]) 

tensor([[ 0.0458, -0.3684,  3.2176,  ...,  0.0740, -0.0588,  2.9572],
        [ 0.3483, -0.3398,  3.8889,  ...,  0.4755, -0.4155,  3.5383],
        [ 0.2563, -0.0092,  3.2534,  ...,  0.5428, -0.0967,  3.3074],
        ...,
        [-0.0443, -0.8832,  4.3851,  ...,  0.2400, -0.8019,  4.2615],
        [ 0.0356, -0.1751,  3.1155,  ...,  0.1627, -0.3184,  2.8636],
        [ 0.3317,  0.0531,  3.9079,  ...,  0.4572,  0.9173,  3.3275]]) 

tensor([[ 0.0458, -0.3684,  3.2176,  ...,  0.0740, -0.0588,  2.9572],
        [ 0.3483, -0.3398,  3.8889,  ...,  0.4755, -0.4155,  3.5383],
        [ 0.2563, -0.0092,  3.2534,  ...,  0.5428, -0.0967,  3.3074],
        ...,
        [-0.0443, -0.8832,  4.3851,  ...,  0.2400, -0.8019,  4.2615],
        [ 0.0356, -0.1751,  3.1155,  ...,  0.1627, -0.3184,  2.8636],
        [ 0.3317,  0.0531,  3.9079,  ...,  0.4572,  0.9173,  3.3275]]) 

torch.Size([50, 75])


In [13]:
# run the model for one epoch
# can be used for both training or validation model
def run_epoch(data_loader, model, criterion, epoch, is_training, optimizer=None):
    if is_training:
        model.train()
        logger_prefix = 'train'
    else:
        model.eval()
        logger_prefix = 'val'

    confusion_matrix = tnt.meter.ConfusionMeter(num_class)
    acc = tnt.meter.ClassErrorMeter(accuracy=True)
    meter_loss = tnt.meter.AverageValueMeter()

    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        label = sample['label']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        input_label_var = Variable(label).type(LongTensor)

        # compute output
        # output_logits: [batch_size, num_class]
        output_logits = model(input_sequence_var)
        #print(output_logits.size())
        #print(output_logits)
        loss = criterion(output_logits, input_label_var)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        meter_loss.add(loss.item())
        acc.add(output_logits.data, input_label_var.data)
        confusion_matrix.add(output_logits.data, input_label_var.data)


    print('%s Epoch: %d  , Loss: %.4f,  Accuracy: %.2f'%(logger_prefix, epoch, meter_loss.value()[0], acc.value()[0]))
    return acc.value()[0]

num_epochs = 10
evaluate_every_epoch = 5
for e in range(num_epochs):
    run_epoch(trLD, model, criterion, e, True, optimizer)
    if e % evaluate_every_epoch == 0:
        run_epoch(valLD, model, criterion, e, False, None)   


train Epoch: 0  , Loss: 2.3045,  Accuracy: 9.15
val Epoch: 0  , Loss: 2.3059,  Accuracy: 9.00
train Epoch: 1  , Loss: 2.3041,  Accuracy: 9.30
train Epoch: 2  , Loss: 2.3044,  Accuracy: 9.75
train Epoch: 3  , Loss: 2.3046,  Accuracy: 9.25
train Epoch: 4  , Loss: 2.3043,  Accuracy: 9.80
train Epoch: 5  , Loss: 2.3039,  Accuracy: 9.35
val Epoch: 5  , Loss: 2.3058,  Accuracy: 9.00
train Epoch: 6  , Loss: 2.3046,  Accuracy: 10.10
train Epoch: 7  , Loss: 2.3036,  Accuracy: 10.25
train Epoch: 8  , Loss: 2.3043,  Accuracy: 9.75
train Epoch: 9  , Loss: 2.3043,  Accuracy: 9.20


## Submit your results on Kaggle

### Train a better model for action recognition!
Now it's your job to experiment with architectures, hyperparameters, loss functions, and optimizers to train a model that achieves better accuracy on the action recognition validation set.


### Testing the model and submit on Kaggle
Testing the model on the testing set and save the results as a .csv file. 
Please submitted the results.csv file generated by predict_on_test() to Kaggle(https://www.kaggle.com/t/934b80879bd741e6ac1967195604d4d9) to see how well your network performs on the test set. 
################ 3rd To Do  (20 points, the highest 3 entries get extra 10 points) ###############


In [ ]:
# Use your best model to generate results on test set.

# generate csv file for test set
def predict_on_test(model, data_loader):
    model.eval() # Put the model in test mode (the opposite of model.train(), essentially)
    results=open('results.csv','w')
    count=0
    results.write('Id'+','+'Class'+'\n')
    for batch_idx, sample in enumerate(data_loader):
        sequence = sample['seq']
        input_sequence_var = Variable(sequence).type(FloatTensor)
        scores = model(input_sequence_var)
        _, preds = scores.data.max(1)
        for i in range(len(preds)):
            results.write(str(count)+','+str(preds[i])+'\n')
            count+=1
    results.close()
    return count

count=predict_on_test(model, tstLD)
print(count)

## Report the performance
################ 4th To Do  (5 points)##################

In this cell, you should write an explanation of what you did (network architecture, optimiziter, learning rate, epochs) and any visualizations or graphs that you make in the process of training and evaluating your network.

Report your Kaggle Performance here: